In [1]:
import pandas
data = pandas.DataFrame({
    'index_action': [],
    'hand_name': [],
    'hand_score': [],
    'last_card': [],
    'action_code': []
}).set_index(['index_action'])
data

,hand_name,hand_score,last_card,action_code
index_action,,,,


In [2]:
data

,hand_name,hand_score,last_card,action_code
index_action,,,,


In [3]:
class Player:
    def __init__(self, name: str):
        self.__name = name
        self.reset()

    def reset(self):
        self.__played_cards = []

    @property
    def name(self):
        return self.__name

    @property
    def played_cards(self):
        return self.__played_cards.copy()

    @property
    def amount_cards_remaining(self):
        return 13 - self.__played_cards.__len__()

    def possible_action_space(self, list_card: list, board_turn_cards: dict, board_turn_cards_owner: str):
        action_space = self.action_space(list_card)
        possible_action_space = {}

        if board_turn_cards['hand_name'] == 'Nothing' or board_turn_cards_owner == self.name:
            return action_space

        list_possible_hand_name = ['Nothing']
        if board_turn_cards['hand_name'] in ['Single', 'Pair', '3_of_a_kind']\
                                            + [f'{k}_straight' for k in range(3,12)]:
            if board_turn_cards['hand_score'] <= 47:
                list_possible_hand_name += [board_turn_cards['hand_name']]
            else:
                if board_turn_cards['hand_name'] == 'Single':
                    list_possible_hand_name += ['Single', '4_of_a_kind']\
                                            + [f'{k}_pairs_straight' for k in [3,4]]
                elif board_turn_cards['hand_name'] == 'Pair':
                    list_possible_hand_name += ['Pair', '4_of_a_kind', '4_pairs_straight']
                else:
                    pass

            for possible_hand_name in list_possible_hand_name:
                if possible_hand_name in action_space.keys():
                    if possible_hand_name == board_turn_cards['hand_name']:
                        temp_list = [action for action in action_space[possible_hand_name] if action['hand_score'] > board_turn_cards['hand_score']]
                        if temp_list.__len__() > 0:
                            possible_action_space[possible_hand_name] = temp_list.copy()
                    else:
                        possible_action_space[possible_hand_name] = action_space[possible_hand_name]

            return possible_action_space
        
        if board_turn_cards['hand_name'] == '3_pairs_straight':
            list_possible_hand_name += [f'{k}_pairs_straight' for k in [3,4]] + ['4_of_a_kind']
        elif board_turn_cards['hand_name'] == '4_of_a_kind':
            list_possible_hand_name += ['4_of_a_kind', '4_pairs_straight']
        elif board_turn_cards['hand_name'] == '4_pairs_straight':
            list_possible_hand_name += ['4_pairs_straight']
        else:
            pass

        for possible_hand_name in list_possible_hand_name:
            if possible_hand_name in action_space.keys():
                if possible_hand_name == board_turn_cards['hand_name']:
                    temp_list = [action for action in action_space[possible_hand_name] if action['hand_score'] > board_turn_cards['hand_score']]
                    if temp_list.__len__() > 0:
                        possible_action_space[possible_hand_name] = temp_list.copy()
                else:
                    possible_action_space[possible_hand_name] = action_space[possible_hand_name]

        return possible_action_space

    def action_space(self, list_card: list):
        action_space = {}
        list_hand_name = ['Nothing', 'Single', 'Pair']\
                        + [f'{k}_of_a_kind' for k in [3,4]]\
                        + [f'{k}_pairs_straight' for k in [3,4]]\
                        + [f'{k}_straight' for k in range(3,12)]

        for hand_name in list_hand_name:
            list_action = self.list_card_hand(list_card, hand_name)
            if list_action.__len__() != 0:
                action_space[hand_name] = list_action.copy()
            
        return action_space

    def list_card_hand(self, list_card: list, hand_name: str):
        list_return = []
        list_card.sort(key=lambda x:x.stt)

        if hand_name == 'Nothing':
            list_return.append({
                'list_card': [],
                'hand_name': 'Nothing',
                'hand_score': -1
            })

        elif hand_name == 'Single':
            list_return += [{
                'list_card': [card],
                'hand_name': 'Single',
                'hand_score': card.stt
            } for card in list_card]

        elif hand_name == 'Pair' or hand_name.endswith('_of_a_kind'):
            n = None
            if hand_name == 'Pair':
                n = 2
            else:
                n = int(hand_name.split('_of_a_kind')[0])

            for i in range(13):
                temp_list = [card for card in list_card if card.score == i]
                if temp_list.__len__() >= n:
                    _temp_list_ = temp_list[:n-1]
                    for j in range(n-1, temp_list.__len__()):
                        list_return.append({
                            'list_card': _temp_list_ + [temp_list[j]],
                            'hand_name': 'Pair' if n == 2 else f'{n}_of_a_kind',
                            'hand_score': temp_list[j].stt
                        })

        elif hand_name.endswith('_pairs_straight'):
            n = int(hand_name.split('_pairs_straight')[0])
            list_score = []
            _list_score_ = []

            for i in range(12):
                temp_list = [card for card in list_card if card.score == i]
                if temp_list.__len__() >= 2:
                    list_score.append(i)
                    _list_score_.append(temp_list)

            list_straight_arr = self.list_straight_subsequence(list_score, n)
            if list_straight_arr.__len__() > 0:
                for straight_arr in list_straight_arr:
                    index_arr = [list_score.index(i) for i in straight_arr]
                    list_max_pair = []
                    max_score_cards = _list_score_[index_arr[n-1]]
                    temp_list = [max_score_cards[0]]
                    for i in range(1, max_score_cards.__len__()):
                        list_max_pair.append(temp_list + [max_score_cards[i]])
                    
                    temp_list = []
                    for i in index_arr[:n-1]:
                        temp_list += _list_score_[i][:2]

                    for pair in list_max_pair:
                        list_return.append({
                            'list_card': temp_list + pair,
                            'hand_name': f'{n}_pairs_straight',
                            'hand_score': pair[1].stt
                        })

        elif hand_name.endswith('_straight'):
            n = int(hand_name.split('_straight')[0])
            list_score = []
            _list_score_ = []

            for i in range(12):
                temp_list = [card for card in list_card if card.score == i]
                if temp_list.__len__() >= 1:
                    list_score.append(i)
                    _list_score_.append(temp_list)

            list_straight_arr = self.list_straight_subsequence(list_score, n)
            if list_straight_arr.__len__() > 0:
                for straight_arr in list_straight_arr:
                    index_arr = [list_score.index(i) for i in straight_arr]
                    max_score_cards = _list_score_[index_arr[n-1]]
                    temp_list = []
                    for i in index_arr[:n-1]:
                        temp_list.append(_list_score_[i][0])

                    for card in max_score_cards:
                        list_return.append({
                            'list_card': temp_list + [card],
                            'hand_name': f'{n}_straight',
                            'hand_score': card.stt
                        })

        return list_return

    def list_straight_subsequence(self, list_int: list, k: int):
        n = list_int.__len__()
        if k <= 2 or n < k:
            return []
        
        list_return = []
        for i in range(0, n-k+1):
            sub_list = list_int[i:i+k]
            if max(sub_list) - min(sub_list) == k-1:
                list_return.append(sub_list)

        return list_return

    def convert_list_card_to_list_dict(self, list_card: list):
        list_return = []
        for card in list_card:
            list_return.append(card.convert_to_dict())

        return list_return

In [4]:
from copy import deepcopy


class Card:
    def __init__(self, stt: int):
        name = ''
        stt %= 52
        score = stt // 4
        if score == 8:
            name = 'Jack'
        elif score == 9:
            name = 'Queen'
        elif score == 10:
            name = 'King'
        elif score == 11:
            name = 'Ace'
        elif score == 12:
            name = '2'
        else:
            name = str(score+3)

        card_type = ''
        b = stt % 4
        if b == 0:
            card_type = 'Spade'
        elif b == 1:
            card_type = 'Club'
        elif b == 2:
            card_type = 'Diamond'
        else:
            card_type = 'Heart'

        name += '_' + card_type

        self.__stt = stt
        self.__name = name
        self.__score = score
        self.__card_type = card_type

    @property
    def stt(self):
        return self.__stt

    @property
    def name(self):
        return self.__name

    @property
    def score(self):
        return self.__score

    @property
    def card_type(self):
        return self.card_type

    def convert_to_dict(self):
        return deepcopy({
            'stt': self.__stt,
            'name': self.__name,
            'score': self.__score,
            'card_type': self.__card_type
        })

In [5]:
An = Player('An')
cards = [Card(i) for i in range(52)]

In [6]:
list_hand_name = ['Nothing', 'Single', 'Pair'] + [f'{k}_of_a_kind' for k in [3,4]] + [f'{k}_straight' for k in range(3,12)] + [f'{k}_pairs_straight' for k in [3,4]]
list_hand_name

['Nothing',
 'Single',
 'Pair',
 '3_of_a_kind',
 '4_of_a_kind',
 '3_straight',
 '4_straight',
 '5_straight',
 '6_straight',
 '7_straight',
 '8_straight',
 '9_straight',
 '10_straight',
 '11_straight',
 '3_pairs_straight',
 '4_pairs_straight']

In [7]:
import warnings
warnings.filterwarnings("ignore")

for hand_name in list_hand_name:
    list_action = An.list_card_hand(cards, hand_name)
    print(f'hand_name: {hand_name}, amount_action: {list_action.__len__()}')
    for action in list_action:
        hand_score = action['hand_score']
        action_code = f'{hand_name}_{hand_score}'
        try:
            last_card = action['list_card'][-1].name
        except:
            last_card = 'Nothing'
        
        if hand_name == '4_of_a_kind' and hand_score == 51:
            continue

        data = data.append({
            'hand_name': hand_name,
            'hand_score': hand_score,
            'action_code': action_code,
            'last_card': last_card
        }, ignore_index=True)

hand_name: Nothing, amount_action: 1
hand_name: Single, amount_action: 52
hand_name: Pair, amount_action: 39
hand_name: 3_of_a_kind, amount_action: 26
hand_name: 4_of_a_kind, amount_action: 13
hand_name: 3_straight, amount_action: 40
hand_name: 4_straight, amount_action: 36
hand_name: 5_straight, amount_action: 32
hand_name: 6_straight, amount_action: 28
hand_name: 7_straight, amount_action: 24
hand_name: 8_straight, amount_action: 20
hand_name: 9_straight, amount_action: 16
hand_name: 10_straight, amount_action: 12
hand_name: 11_straight, amount_action: 8
hand_name: 3_pairs_straight, amount_action: 30
hand_name: 4_pairs_straight, amount_action: 27


In [9]:
data.to_csv('action_space.csv')

In [10]:
data

,hand_name,hand_score,last_card,action_code
0,Nothing,-1.0,Nothing,Nothing_-1
1,Single,0.0,3_Spade,Single_0
2,Single,1.0,3_Club,Single_1
3,Single,2.0,3_Diamond,Single_2
4,Single,3.0,3_Heart,Single_3
...,...,...,...,...
398,4_pairs_straight,42.0,King_Diamond,4_pairs_straight_42
399,4_pairs_straight,43.0,King_Heart,4_pairs_straight_43
400,4_pairs_straight,45.0,Ace_Club,4_pairs_straight_45
401,4_pairs_straight,46.0,Ace_Diamond,4_pairs_straight_46
